# Search through gnps libraries to select dataset

Starting from gnps public libraries (speclibs folders, version from 5 July 2019), we here create one large MGF file with all spectra.
In addition Smiles or Inchi's are added from metadata file.

In [3]:
# Import
import json
import os

In [4]:
# Locations
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA"
PATH_MS_DATA = ROOT + "\\Data\\GNPS_all\\"
PATH_SAVE_MODEL = ROOT + "\\Spec2Vec\\models_trained\\"
PATH_SAVE_DATA = ROOT + "\\Spec2Vec\\data\\"
PATH_SPEC2VEC = ROOT + "\\Spec2Vec\\"

## Importing additional metadata

In [5]:
json_file = PATH_MS_DATA + 'gnpslibraryjson_filtered.json'

with open(json_file, 'rb') as f:
    data = json.load(f)

In [6]:
print("Metadata available for", len(data), "spectra.")

Metadata available for 77253 spectra.


In [5]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry

## Briefly inspect metadata

In [6]:
keys = []
for key in gnps_data.keys():
    keys.append(key)

In [8]:
gnps_data[keys[0]]

{'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
 'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
 'LIBRARY_QUALITY': '1',
 'GNPSID': 'CCMSLIB00000001548',
 'COMPOUND_NAME': 'Hoiamide B',
 'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'}

## Functions

In [1]:
def check_spectrum_lines(spectrum):
    """ Check if desired metadata is present for spectrum.
    """
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    """ Add metadata to spectrum (taken from json file, see above).
    """
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename, matching=True):
    """ Collect spectra in file (line by line). Check if matching metadata is available.
    
    matching: bool
        If matching = True than only include spectra for which desired metadata is present.
        For matching = False, include all spectra.
    """
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            
            if line.startswith('END ION'):
                
                if check_spectrum_lines(spectrum): # Check if we have structural info
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
                else: 
                    if not matching: # then include all spectra
                        matched_spectra.append(spectrum)
    return matched_spectra

## Create MGF for all spectra (also non matched ones)

In [25]:
path = PATH_MS_DATA + 'speclibs\\'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f), matching=False)
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))

BILELIB19.mgf 177
CASMI.mgf 745
DEREPLICATOR_IDENTIFIED_LIBRARY.mgf 1124
GNPS-COLLECTIONS-MISC.mgf 1170
GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf 1246
GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf 1899
GNPS-EMBL-MCF.mgf 2484
GNPS-FAULKNERLEGACY.mgf 2611
GNPS-LIBRARY.mgf 7394
GNPS-NIH-CLINICALCOLLECTION1.mgf 7771
GNPS-NIH-CLINICALCOLLECTION2.mgf 7966
GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf 9233
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf 11096
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf 16892
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf 18352
GNPS-NIST14-MATCHES.mgf 24115
GNPS-PRESTWICKPHYTOCHEM.mgf 24258
GNPS-SELLECKCHEM-FDA-PART1.mgf 26646
GNPS-SELLECKCHEM-FDA-PART2.mgf 27302
HMDB.mgf 29537
LDB_NEGATIVE.mgf 29763
LDB_POSITIVE.mgf 29846
MASSBANK.mgf 41845
MASSBANKEU.mgf 43337
MIADB.mgf 43509
MMV_NEGATIVE.mgf 43556
MMV_POSITIVE.mgf 43666
MONA.mgf 92907
PNNL-LIPIDS-NEGATIVE.mgf 109049
PNNL-LIPIDS-POSITIVE.mgf 139631
RESPECT.mgf 146743
SUMNER.mgf 147647


In [26]:
len(collected_spectra)

147647

In [27]:
import sys

file_mgf = PATH_MS_DATA + 'all_gnps_191015.mgf'
with open(file_mgf, 'w') as f:
    for c in collected_spectra[:]:
        for l in c:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=15-formyl-16-hydroxy-1,2,14,17,17-pentamethyl-8-(prop-1-en-2-yl)pentacyclo[11.7.0.0�,�?.0?,?.0�?,�?]icosane-5-carboxylic acid

TITLE=(1R,2S,5S,8R)-7,7-dimethyl-6-methylidenetricyclo[6.2.1.0�,?]undecane-2-carboxylic acid

TITLE=(1R,2S,5S,8R)-7,7-dimethyl-6-methylidenetricyclo[6.2.1.0�,?]undecane-2-carboxylic acid

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy}-6-hydroxy-8-methyl-9,10-dioxatetracyclo[4.3.1.0�,?.0�,?]decan-2-yl]methyl benzoate

TITLE=[(1S,3R,5R,6S,8S)-3-{[(2S,3R,4S,5S,6R)-6-[(benzoyloxy)methyl]-3,4,5-trihydroxyoxan-2-yl]oxy

TITLE=15-(carbamoylmethyl)-10,11,23-trihydroxy-18-(3-methyl-2-oxopentanamido)-9,14,17-trioxo-N-[(1Z)-prop-1-en-1-yl]-8,13,16-triazatetracyclo[18.3.1.0�,?.0?,�?]tetracosa-1(23),2(7),3,5,20(24),21-hexaene-12-carboxamide

TITLE=15-(carbamoylmethyl)-10,11,23-trihydroxy-18-(3-methyl-2-oxopentanamido)-9,14,17-trioxo-N-[(1Z)-prop-1-en-1-yl]-8,13,16-triazatetracyclo[18.3.1.0�,?.0?,�?]tetracosa-1(23),2(7),3,5,20(24),21-hexaene-12-carboxamide

TITLE=15-(carbamoylmethyl)-10,11,23-trihydroxy-18-(3-methyl-2-oxopentanamido)-9,14,17-trioxo-N-[(1Z)-prop-1-en-1-yl]-8,13,16-triazatetracyclo[18.3.1.0�,?.0?,�?]tetracosa-1(23),2(7),3,5,20(24),21-hexaene-12-carboxamide

TITLE=15-(carbamoylmethyl)-10,11,23-trihydroxy-18-(3-methyl-2-oxopentanamido)-9,14,17-trioxo-N-[(1Z)-prop-1-en-1-yl]-8,13,16-triazatetracyclo[18.3.1.0�,?.0?,�?]tetracosa-1(23),2(7),3,5,20(24),21-hexaene-12-carboxamide

TITLE=2-{[(17Z)-17-ethylidene-14-(1-hydroxyethyl)-27-(2-hydroxypropan-2-yl)-20,33-dimethyl-24,30,37,40-tetramethylidene-12,1

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4

...

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethy

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]docosan-11-yl acetate

TITLE=(2R,7S,13R,14R,16S,19R,20S)-19-(furan-3-yl)-9,9,13,20-tetramethyl-5,12,17-trioxo-4,8,15,18-tetraoxahexacyclo[11.9.0.0�,?.0�,�?.0�?,�?.0�?,�?]doc

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[9.5.0.0�,?.0?,�?.0�?,�?]hexadec-8-en-14-yl dodecanoate

TITLE=(1R,2S,4R,5S,6S,11R,12R,14S,16R)-5,6,11-trihydroxy-4-(hydroxymethyl)-8,12,15,15-tetramethyl-7-oxo-3-oxapentacyclo[

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione

TITLE=4,5-dihydroxy-4,5,6-trimethyl-2,8-dioxa-13-azatricyclo[8.5.1.0��,�?]hexadec-10-ene-3,7-dione



TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethyl

## Replace charge values of "-1+" with "-1"

In [28]:
input_file = PATH_MS_DATA + 'MS_data_allGNPS_191015.mgf'
output_file = open(ROOT + "\\Data\\GNPS_all\\" + "MS_data_allGNPS_191015_corrected.mgf", 'w')

with open(input_file, 'r') as file:
    for line in file:
        output_file.write(line.replace('-1+', '-1'))

In [29]:
output_file.close()